# Chinese AD9910 PCB → Arduino

### Hardware setup

Connect the PCB to the arduino as follows:

![](connections.png)
    
Since an Arduino is used as a serial-to-SPI link to interface between the computer and the AD9910, it has to be programmed appropriately. See [`SPI_test.ino`](https://github.com/js216/Arduino-codes/blob/master/SPI_test/SPI_test.ino) for example code that is assumed in the following. Other Arduino-like boards might necessitate other pin assignments; see the [official website](https://www.arduino.cc/en/reference/SPI) for details, and adjust the provided [Arduino code](https://github.com/js216/Arduino-codes/blob/master/SPI_test/SPI_test.ino) as appropriate.
    
### Software implementation

Import packages:

In [1]:
import pyvisa

Connect to the Arduino:

In [9]:
rm = pyvisa.ResourceManager()
a = rm.open_resource('ASRL13::INSTR')

Write and test the function that prints the contents of one register (specified in the Arduino source code which one; currently 0x02):

In [10]:
def read_one_register(arduino):
    # send the read command
    arduino.write_raw(b"1")
    
    # print Arduino's return text
    return_text = arduino.read(), arduino.read()
    for t in return_text:
        print(t, end='')
        
read_one_register(a)

Reading from register 2:   
  00011111,00111111,01000000,00000000,


Write and test a function to read all 23 registers:

In [11]:
def read_registers(arduino):
    print(arduino.query('r'), end='')
    for i in range(23):
        print(arduino.read(), end='')
        
read_registers(a)

Reading register contents:
   0: 00000000,00000000,00000000,00000000,
   1: 00000000,01000000,00001000,00100000,
   2: 00011111,00111111,01000000,00000000,
   3: 00000000,00000000,01111111,01111111,
   4: 11111111,11111111,11111111,11111111,
   5: 00000000,00000000,00100001,00000000,01000001,00100000,
   6: 00000000,00000000,01000001,00001000,00000000,00000000,
   7: 00000000,00000000,00000000,00000000,
   8: 00000000,00000000,
   9: 00000000,00000000,00000000,00000000,
   A: 00000000,00000000,00000000,00000000,
   B: 01000000,00000010,11100101,00000100,00100010,11000001,00010000,00111110,
   C: 01100000,00000000,00000000,00000001,00100000,00101000,00000000,10000011,
   D: 00001010,00000110,01100011,00000000,
   E: 00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,
   F: 00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,
  10: 00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,
  11: 00000000,00000000,00000000,00000000,00

Refer to the [AD9910 datasheet](https://www.analog.com/media/en/technical-documentation/data-sheets/AD9910.pdf), page 49ff, for a description of the register contents.

Write functions to reset the AD9910, and to write register contents:

In [12]:
def reset(arduino):
    print(arduino.query('0'))
    
def write_register(arduino, reg, data_list, printing=False):
    # send the write command
    arduino.write_raw(
        b"w" \
          + bytes(bytearray([reg])) \
          + bytes(bytearray(data_list))
      )
    
    # print Arduino's return text
    return_text = arduino.read(), arduino.read(), arduino.read(), arduino.read(),  arduino.read(),  arduino.read()
    if printing:
        for t in return_text:
            print(t, end='')

Test writing to one of the registers:

In [13]:
write_register(
    a,
    0x02,
    [0b00011111,0b00111111,0b11000000,0b00000000,],
    printing=True
  )

Reading from register 2:   
  00011111,00111111,01000000,00000000,
Writing to register 2:   
  00011111,00111111,11000000,00000000,
Reading from register 2:   
  00011111,00111111,11000000,00000000,


The following writes the register values that cause the AD9910 to output an 85 MHz sine wave for a 1 GHz clock input.

Note in particular the MSB of the second least significant byte in register 0x02, "REFCLK input divider bypass", which, when set to 1, disables the internal clock divider.

![](CFR3.png)

The amplitude, phase, and frequency settings of profile 0 are written in the 0x0E register.

In [29]:
write_register(a, 0x00, [0b00000000,0b00000000,0b00000000,0b00000000,])
write_register(a, 0x01, [0b00000000,0b01000000,0b00001000,0b00100000,])
write_register(a, 0x02, [0b00011111,0b00111111,0b11000000,0b00000000,])
write_register(a, 0x03, [0b00000000,0b00000000,0b01111111,0b01111111,])
write_register(a, 0x04, [0b11111111,0b11111111,0b11111111,0b11111111,])
write_register(a, 0x05, [0b11101001,0b00100000,0b01100100,0b10001001,0b11000100,0b11100000,])
write_register(a, 0x06, [0b10000111,0b00101100,0b00000101,0b10101001,0b10000000,0b00101001,])
write_register(a, 0x07, [0b00000000,0b00000000,0b00000000,0b00000000,])
write_register(a, 0x08, [0b00000000,0b00000000,])
write_register(a, 0x09, [0b00000000,0b00000000,0b00000000,0b00000000,])
write_register(a, 0x0A, [0b00000000,0b00000000,0b00000000,0b00000000,])
write_register(a, 0x0B, [0b10100001,0b10011001,0b11000100,0b00011010,0b00011001,0b00101110,0b00000011,0b11000100,])
write_register(a, 0x0C, [0b11000100,0b10101001,0b10110000,0b10000101,0b00000011,0b11000000,0b10000011,0b10110000,])
write_register(a, 0x0D, [0b01001110,0b11010000,0b00000000,0b00011100,])
write_register(a, 0x0E, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x0F, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x10, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x11, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x12, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x13, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x14, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x15, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x16, [0b00111000,0b11010000,0b01010011,0b11000111,])

Close connection to the Arduino:

In [ ]:
a.close()

### Parallel data input

In order to enable controlling the AD9910 via the 18-bit parallel port, the Parallel data port enable bit in the least significant byte of register `CFR2` (0x01) has to be 1:

![](CFR2.png)

In addition, a high voltage level has to be applied to the `TxENABLE` pin of the AD9910 (`TEN` on the PCB):

![](TEN.png)

In [36]:
write_register(
    a,
    0x01,
    [0b00000000,0b01000000,0b00001000,0b00110000,],
    printing=True
  )

Reading from register 1:   
  00000000,01000000,00001000,00100000,
Writing to register 1:   
  00000000,01000000,00001000,00110000,
Reading from register 1:   
  00000000,01000000,00001000,00110000,
